Python for Data Manipulation

Loading Data from SQL Databases

In [13]:
import sqlite3

In [4]:
db = sqlite3.connect('players.db')

In [5]:
cr = db.cursor()

In [ ]:
cr.execute("""
    CREATE TABLE players (
        player_id NUMBER constraint players_pk primary key,
        player_name VARCHAR2(100) not null constraint players_uk1 unique,
        player_age NUMBER not null,
        passes_id NUMBER constraint players_fk1 REFERENCES passing_stats (passes_id),
        defensive_id NUMBER constraint players_fk2 REFERENCES defensive_stats (defensive_id),
        attacking_id NUMBER constraint players_fk3 REFERENCES attacking_stats (attacking_id)
    )
""")

In [ ]:
cr.execute("""
    CREATE TABLE passing_stats (
         passes_id NUMBER constraint passing_stats_pk primary key,
         all_passes NUMBER constraint passing_stats_uk1 unique,
         passes_acc NUMBER not null,
         forward_psses NUMBER not null,
         side_passes NUMBER not null,
         passes_to_final_third NUMBER,
         long_passes NUMBER not null
    )      
""")

In [ ]:
cr.execute("""
    CREATE TABLE defensive_stats (
         defensive_id NUMBER constraint defensive_stats_pk primary key,
         all_blocks NUMBER constraint defensive_stats_uk1 unique,
         interceptions NUMBER not null,
         duals NUMBER not null,
         tackles NUMBER not null
)
""")

In [ ]:
cr.execute("""
CREATE TABLE attacking_stats (
  attacking_id NUMBER constraint attacking_stats_pk primary key,
  goals NUMBER not null,
  assists NUMBER not null,
  xG NUMBER not null,
  xA NUMBER not null,
  shots NUMBER not null,
  creating_chances NUMBER not null
)
""")

In [26]:
cr.execute("""INSERT INTO passing_stats VALUES (401, 792, 63, 45, 877, 245, 25);
""")

In [ ]:
cr.execute("""
INSERT INTO passing_stats VALUES (2, 2504, 79, 951, 1203, 479, 688);
""")

In [ ]:
cr.execute("""
INSERT INTO passing_stats VALUES (3, 2217, 71, 711, 1008, 358, 455);
""")

In [ ]:
cr.execute("""
INSERT INTO passing_stats VALUES (4, 2501, 65, 522, 1500, 501, 570);
""")

In [ ]:
cr.execute("""
INSERT INTO passing_stats VALUES (5, 2110, 66, 455, 1200, 455, 704);
""")

In [ ]:
cr.execute("""
INSERT INTO attacking_stats VALUES (1, 41, 11, 55, 18, 289, 307);
""")

In [ ]:
cr.execute("""
INSERT INTO attacking_stats VALUES (2, 37, 26, 40, 31, 385, 204);
""")

In [ ]:
cr.execute("""
INSERT INTO attacking_stats VALUES (3, 25, 19, 35, 28, 259, 275);
""")

In [ ]:
cr.execute("""
INSERT INTO attacking_stats VALUES (4, 22, 14, 27, 15, 299, 310);
""")

In [ ]:
cr.execute("""
INSERT INTO attacking_stats VALUES (5, 29, 17, 31, 24, 282, 267);
""")

In [21]:
cr.execute("""
INSERT INTO defensive_stats VALUES (2, 51, 69, 25, 31);
""")

In [22]:
cr.execute("""
INSERT INTO defensive_stats VALUES (3, 45, 44, 19, 32);
""")

In [23]:
cr.execute("""
INSERT INTO defensive_stats VALUES (4, 56, 96, 75, 44);
""")

In [24]:
cr.execute("""
INSERT INTO defensive_stats VALUES (5, 47, 66, 27, 30);
""")

In [ ]:
cr.execute("""
INSERT INTO players VALUES (1, 'Halland', 23, 1, 1, 1);
""")

In [ ]:
cr.execute("""
INSERT INTO players VALUES (2, 'Salah', 31, 2, 2, 2);
""")

In [ ]:
cr.execute("""
INSERT INTO players VALUES (3, 'Saka', 23, 3, 3, 3);
""")

In [ ]:
cr.execute("""
INSERT INTO players VALUES (4, 'Fernandez', 28, 4, 4, 4);""")

In [ ]:
cr.execute("""
INSERT INTO players VALUES (5, 'Son', 26, 5, 5, 5);
""")

In [ ]:
import pandas as pd

In [18]:
conn = sqlite3.connect('players.db')

In [19]:
df = pd.read_sql_query('SELECT * FROM defensive_stats', conn)

In [25]:
df = pd.read_sql_query('SELECT * FROM defensive_stats', conn)
print (df)

Empty DataFrame
Columns: [defensive_id, all_blocks, interceptions, duals, tackles]
Index: []


Building a Robust Data Loading Pipeline for Data Analysis

Step 1: Load Data from Multiple Sources

In [32]:
csv_df = pd.read_csv('epl2021.csv')

sql_df = pd.read_sql_query('SELECT * FROM players', conn)

# Merge all datasets

all_data = pd.concat([csv_df, sql_df], ignore_index = True)

Step 2: Handle Missing Values

In [35]:
# Remove rows with more than 50% missing values
cleaned_data = all_data.dropna(thresh=len(all_data.columns) * 0.5)

In [39]:
# Fill remaining missing values with the mean for numerical columns
cleaned_data['Goals'].fillna(cleaned_data['Goals'].mean(), inplace = True)

Step 3: Validate Data Types

In [40]:
# Ensure Goals column as int
cleaned_data['Goals'] = cleaned_data['Goals'].astype(int)

Step 4: Save Cleaned Data

In [41]:
# Save the cleaned data to a new CSV file

cleaned_data.to_csv('cleaned_players.csv', index=False)

In [48]:
import sqlite3
conn = sqlite3.connect('players.db')

In [53]:
import threading

thread_local = threading.local()

def get_db_connection():
    if not hasattr(thread_local, "conn"):
        thread_local.conn = sqlite3.connect('cleaned_players.db')
    return thread_local.conn

def save_to_db(df):
    conn = get_db_connection()
    df.to_sql('players_stats', conn, if_exists='replace', index=False)

save_to_db(cleaned_data)

Step 5: Automating the Pipeline

In [1]:
import pandas as pd
import sqlite3

def load_data():
    conn = sqlite3.connect('players.db')
    csv_df = pd.read_csv('epl2021.csv')
    sql_df = pd.read_sql_query('SELECT * FROM players', conn)
    conn.close() 
    return pd.concat([csv_df, sql_df], ignore_index=True)

def clean_data(data):
    data = data.dropna(thresh=len(data.columns) * 0.5)
    if 'Goals' in data.columns:
        data['Goals'].fillna(data['Goals'].mean(), inplace=True)
    return data 

def save_data(data):
    data.to_csv('cleaned_players.csv', index=False)

    conn = sqlite3.connect('players.db')
    data.to_sql('cleaned_players', conn, if_exists='replace', index=False)
    conn.close() 

raw_data = load_data()
cleaned_data = clean_data(raw_data)
save_data(cleaned_data)

In [2]:
import pandas as pd
import sqlite3

def load_data():
    conn = sqlite3.connect('players.db')
    csv_df = pd.read_csv('epl2021.csv')
    sql_df = pd.read_sql_query ('SELECT * FROM players', conn)
    conn.close()
    return pd.concat([csv_df, sql_df], ignore_index = True)

def clean_data(data):
    data = data.dropna(thresh = len(data.columns) * 0.5)
    if 'Goals' in data.columns:
        data['Goals'].fillna(data['Goals'].mean(), inplace = True)
    return data

def save_data(data):
    data.to_csv('cleaned_players_csv.csv', index = False)
    conn = sqlite3.connect('players.db')
    data.to_sql('cleaned_players_db', conn, if_exists = 'replace', index = False)
    conn.close()

raw_data = load_data()
cleaned_data = clean_data(raw_data)
save_data(cleaned_data)